In [1]:
import glob
import json
import sys

from fashion_clip.fashion_clip import FashionCLIP
import numpy as np
from sklearn.metrics import *
import pandas as pd
import os

sys.path.append("C:/Users/yuuta/Documents/fashion")
from utils.util import get_progress_percent, is_target_category

fclip = FashionCLIP("fashion-clip")

# ターゲットのディレクトリ
KIND = "train"
BATCH_SIZE = 512

coordinates_file = glob.glob(
    f"C:/Users/yuuta/Documents/fashion/data/{KIND}/**/*_new.json"
)

print("size: ", len(coordinates_file))


c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


size:  162975


In [2]:
images = []
item_set = set()
for i, fp in enumerate(coordinates_file):
    json_dict = pd.read_json(fp, encoding="shift-jis")
    if i % 1000 == 0:
        print(get_progress_percent(i, len(coordinates_file)), end="")
    for item in filter(is_target_category, json_dict["items"]):
        item_id = item["itemId"]
        if item_id in item_set:
            continue
        item_set.add(item_id)
        images.append(f"C:/Users/yuuta/Documents/fashion/data/images/{item_id}.jpg")

print(len(images))



【                    】

KeyboardInterrupt: 

In [3]:
image_embeddings = fclip.encode_images(images, batch_size=BATCH_SIZE)
image_embeddings = image_embeddings / np.linalg.norm(
    image_embeddings, ord=2, axis=-1, keepdims=True
)
print("image fin")

187it [10:30,  3.37s/it]                         

image fin


In [4]:
# 6795055 in item_set

In [5]:
# for fp in coordinates_file:
#     json_dict = pd.read_json(fp, encoding="shift-jis")
#     coordinate_attributes = ""
#     for item in filter(is_target_category, json_dict["items"]):
#         item_id = item["itemId"]
#         if item_id not in item_set:
#             print(item_id, ' not in')
#             break

In [6]:
category = [
    "tops",
    "bottoms",
    "shoes",
    "others",
]
materials = [
    "Cotton",
    "Polyester",
    "Rayon",
    "Linen",
    "Wool",
    "Silk",
    "Nylon",
    "Polyurethane",
    "Denim",
    "Spandex",
]
colors = [
    "Black",
    "White",
    "Gray",
    "Navy",
    "Blue",
    "Red",
    "Green",
    "Brown",
    "Beige",
    "Pink",
    "Yellow",
    "Orange",
    "Purple",
    "Lavender",
    "Teal",
    "Turquoise",
    "Magenta",
    "Olive",
    "Maroon",
    "Charcoal",
]
patterns = [
    "crochet",
    "camouflage",
    "floral",
    "geo",
    "horizontal_striped",
    "lace",
    "leopard",
    "plaid",
    "paisley",
    "plain",
    "polka",
    "tribal",
    "vertical",
    "zebra",
]

category_embeddings = fclip.encode_text(category, batch_size=BATCH_SIZE)
material_embeddings = category_embeddings / np.linalg.norm(
    category_embeddings, ord=2, axis=-1, keepdims=True
)

material_embeddings = fclip.encode_text(materials, batch_size=BATCH_SIZE)
material_embeddings = material_embeddings / np.linalg.norm(
    material_embeddings, ord=2, axis=-1, keepdims=True
)

color_embeddings = fclip.encode_text(colors, batch_size=BATCH_SIZE)
color_embeddings = color_embeddings / np.linalg.norm(
    color_embeddings, ord=2, axis=-1, keepdims=True
)

patterns_embeddings = fclip.encode_text(patterns, batch_size=BATCH_SIZE)
patterns_embeddings = patterns_embeddings / np.linalg.norm(
    patterns_embeddings, ord=2, axis=-1, keepdims=True
)

predicted_materials = material_embeddings.dot(image_embeddings.T)
predicted_materials = [materials[k] for k in np.argmax(predicted_materials, axis=0)]

print("materials fin")

predicted_colors = color_embeddings.dot(image_embeddings.T)
predicted_colors = [colors[k] for k in np.argmax(predicted_colors, axis=0)]

print("colors fin")

predicted_patterns = patterns_embeddings.dot(image_embeddings.T)
predicted_patterns = [patterns[k] for k in np.argmax(predicted_patterns, axis=0)]

print("patterns fin")

predicted_cateogories = category_embeddings.dot(image_embeddings.T)
predicted_cateogories = [category[k] for k in np.argmax(predicted_cateogories, axis=0)]

print("category fin")

1it [00:00, 45.45it/s]                           
1it [00:00,  9.42it/s]                            
1it [00:00,  4.90it/s]                            
1it [00:00, 30.30it/s]                            

materials fin
colors fin
patterns fin
category fin


In [7]:
item_id_to_attributes = {}
for image_path, m, c, p, category in zip(
    images,
    predicted_materials,
    predicted_colors,
    predicted_patterns,
    predicted_cateogories,
):
    item_id = image_path.split("/")[-1][:-4]
    item_id_to_attributes[item_id] = [
        category + "_" + m,
        category + "_" + c,
        category + "_" + p,
    ]

with open(
    f"C:/Users/yuuta/Documents/fashion/data/attributes_{KIND}.json",
    "w",
    encoding="utf-8",
) as f:
    json.dump(item_id_to_attributes, f)